# Seldon Core Setup

## Prerequisites

You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [A running Kubernetes cluster with kubectl authenticated](https://kubernetes.io/docs/setup/)
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

## Setup Cluster

In [2]:
!kubectl create namespace seldon

zsh:1: command not found: kubectl


In [3]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_airy-berm-306315_us-central1-c_seldon" modified.


## Install Seldon Core

Follow the [Seldon Core Install documentation](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html).

In [4]:
!kubectl create namespace seldon-system

namespace/seldon-system created


### If using Ambassador:

In [5]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set ambassador.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

NAME: seldon-core
LAST DEPLOYED: Tue Jan 18 20:32:57 2022
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


### If using Istio:

In [6]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set istio.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

NAME: seldon-core
LAST DEPLOYED: Tue Jan 18 16:57:54 2022
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


Check all services are running before proceeding.

### Wait for rollout to finish

In [6]:
!kubectl rollout status deploy/seldon-controller-manager -n seldon-system

deployment "seldon-controller-manager" successfully rolled out


## Install Ingress

### Ambassador

[Ambassador install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#install-ambassador)

Ambassador Edge Stack:

In [ ]:
!helm repo add datawire https://www.getambassador.io

In [ ]:
!helm repo update

In [ ]:
!helm install ambassador datawire/ambassador \
    --set image.repository=docker.io/datawire/ambassador \
    --set crds.keep=false \
    --namespace seldon-system

Ambassador API Gateway (Emissary Ingress):

In [7]:
!helm repo add datawire https://www.getambassador.io

"datawire" already exists with the same configuration, skipping


In [8]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "datawire" chart repository
...Successfully got an update from the "prometheus-community" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈Happy Helming!⎈


In [9]:
!helm install ambassador datawire/ambassador \
    --set image.repository=docker.io/datawire/ambassador \
    --set crds.keep=false \
    --set enableAES=false \
    --namespace seldon-system

W0118 20:36:57.854850   12355 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0118 20:36:58.010172   12355 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0118 20:36:58.403867   12355 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0118 20:36:58.568575   12355 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefinition
W0118 20:36:59.088583   12355 warnings.go:70] apiextensions.k8s.io/v1beta1 CustomResourceDefinition is deprecated in v1.16+, unavailable in v1.22+; use apiextensions.k8s.io/v1 CustomResourceDefini

Check all services are running before proceeding.

In [10]:
!kubectl rollout status deployment.apps/ambassador

Error from server (NotFound): deployments.apps "ambassador" not found


For Ambassador Edge Stack:

**Note:** Absent configuration, Ambassador Edge Stack will attempt to use TLS with a self-signed cert.  Either [set up TLS](https://www.getambassador.io/docs/edge-stack/latest/howtos/tls-termination/) or ignore certificate validation (i.e. `curl -k`).

In [ ]:
!kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8443

For Ambassador API Gateway:

In [ ]:
!kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8080

*or*

### Istio

[Istio install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#ingress-support)

**Note:** Remember to add ```--set istio.enabled=true``` flag when installing Seldon Core with Istio Ingress.